# Inferencia y Comparacion: Antes vs Despues del Fine-tuning

Comparamos las imagenes generadas por Stable Diffusion v1-4 antes y despues
del fine-tuning con el dataset Old Book Illustrations.

In [ ]:
from diffusers import StableDiffusionPipeline
from diffusers import UNet2DConditionModel
from PIL import Image
import matplotlib.pyplot as plt
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model_name = "CompVis/stable-diffusion-v1-4"
prompt = "an illustration of a ship sailing through a stormy sea"

print(f"Device: {device}")
print(f"Prompt: {prompt}")

## 1. Imagen ANTES del fine-tuning

In [ ]:
# Cargar modelo base
pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name,
).to(device)

image_before = pipe(prompt).images[0]
image_before.save("../generated/before_finetuning.png")

image_before

In [ ]:
# Liberar memoria
del pipe
if device == "cuda":
    torch.cuda.empty_cache()

## 2. Imagen DESPUES del fine-tuning

In [ ]:
# Cargamos la UNet finetuneada:
finetuned_unet = UNet2DConditionModel.from_pretrained("../outputs/finetuned-model")
finetuned_unet.to(device)

print('Modelo finetuneado cargado correctamente!')

In [ ]:
# Cargamos el modelo pre-entrenado pero sustituyendo la UNet por la nuestra:
pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name,
    unet=finetuned_unet,
).to(device)

image_after = pipe(prompt).images[0]
image_after.save("../generated/after_finetuning.png")

image_after

## 3. Comparacion side-by-side

In [ ]:
# Cargar ambas imagenes (por si se ejecuta esta celda independientemente)
image_before = Image.open("../generated/before_finetuning.png")
image_after = Image.open("../generated/after_finetuning.png")

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

axes[0].imshow(image_before)
axes[0].set_title('ANTES del fine-tuning', fontsize=14)
axes[0].axis('off')

axes[1].imshow(image_after)
axes[1].set_title('DESPUES del fine-tuning', fontsize=14)
axes[1].axis('off')

plt.suptitle(f'Prompt: "{prompt}"', fontsize=12, style='italic')
plt.tight_layout()
plt.savefig("../generated/comparison.png", dpi=150, bbox_inches='tight')
plt.show()

print("Comparacion guardada en ../generated/comparison.png")

## 4. Prueba con prompts adicionales

In [ ]:
# Generar con otros prompts para ver el efecto del fine-tuning
test_prompts = [
    "a medieval knight riding a horse",
    "a forest with ancient trees and a river",
    "a portrait of an old man with a long beard",
]

fig, axes = plt.subplots(len(test_prompts), 1, figsize=(7, 7 * len(test_prompts)))

for ax, test_prompt in zip(axes, test_prompts):
    img = pipe(test_prompt).images[0]
    ax.imshow(img)
    ax.set_title(test_prompt, fontsize=12)
    ax.axis('off')

plt.suptitle('Imagenes generadas con modelo fine-tuneado', fontsize=14)
plt.tight_layout()
plt.show()